In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys, subprocess
from pathlib import Path
_root_dir = Path(subprocess.check_output(['git','rev-parse','--show-toplevel']).strip().decode("utf-8"))
os.chdir(_root_dir)
sys.path.append(str(_root_dir / "centernet-master/src"))
import _init_paths

## Imports

In [ ]:
import os
import cv2
from opts import opts
from detectors.detector_factory import detector_factory
from lib.datasets.dataset_factory import get_dataset
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from h4dlib.config import h4dconfig

## Some helper functions:

In [ ]:
def demo(opt):
    image_ext = ['jpg', 'jpeg', 'png', 'webp']
    video_ext = ['mp4', 'mov', 'avi', 'mkv']
    time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge']

    os.environ["CUDA_VISIBLE_DEVICES"] = opt.gpus_str
    Detector = detector_factory[opt.task]
    detector = Detector(opt)

    if isinstance(opt.demo, list):
        image_names = opt.demo
    elif os.path.isdir(opt.demo):
        image_names = []
        ls = os.listdir(opt.demo)
        for file_name in sorted(ls):
            ext = file_name[file_name.rfind(".") + 1 :].lower()
            if ext in image_ext:
                image_names.append(os.path.join(opt.demo, file_name))
    else:
        image_names = [opt.demo]

    for image_name in image_names:
        print("Running detection on image: ", image_name)
        ret = detector.run(image_name)
        time_str = ""
        for stat in time_stats:
            time_str = time_str + "{} {:.3f}s |".format(stat, ret[stat])
        print(time_str)

def show_options(opt):
    for k in dir(opt):
        if not k.startswith("_"):
            print("    ", k, "->", getattr(opt, k))

## Configure the folder and COCO API object for images on which to run detections:

In [ ]:
# Split ("train" | "val"):
split = "val"

# Setup the path to the demo image(s):
demo_image_path = h4dconfig.DATA_DIR / f"Xview/coco_chipped/{split}/"

# num images to run detection on:
num_demo_images = 10

In [ ]:
# Load dataset as COCO so we can filter demo images to only ones that have annotations:
dataDir = h4dconfig.DATA_DIR / 'Xview/coco_chipped'
annFile = dataDir / f"xview_coco_v2_{split}_chipped.json"

print("demo_image_path: ", demo_image_path)
print("xviewcoco_dir: ", dataDir)
print("annFile: ", annFile)

coco = COCO(annFile)

## Output some dataset stats:

In [ ]:
imgs = coco.dataset["images"]
anns = coco.dataset["annotations"]

print("Total images: ", len(imgs))
print("Total images with annotations: ", len(coco.imgToAnns.keys()))
print("Total annotations: ", len(anns))
print("Average annotations per image (ignoring images w/o any anns): ", np.average([len(anns) for image_id, anns in coco.imgToAnns.items()]))

## Set up list of possible demo images from the specified folder:

In [ ]:
all_images = list(demo_image_path.glob("*.jpg"))

# Filter to images that have annotations
images_with_anns = {coco.imgs[img_id]["file_name"] for img_id in coco.imgToAnns.keys()}
all_images_with_anns = [img for img in all_images if img.name in images_with_anns]
print(f"# images: {len(all_images)}, #images w/ anns: {len(all_images_with_anns)}")

# Choose here if you want to only use images that have anns:
use_anns_only = True
if use_anns_only:
    all_images = all_images_with_anns

## Configure Detection Options:

In [ ]:
args = "ctdet --exp_id xview_dla_512 --dataset xview".split()
opt = opts().h4d_init(args=args)
opt.load_model = Path("./centernet-master/exp/ctdet/xview_dla_512/model_best.pth").resolve()
opt.debug = 3

print("\nOptions:")
show_options(opt)

## Randomly select `num_demo_images` to run detections on:

In [ ]:
# Pick `num_demo_images` random images to run detection on:
demo_image_indexes = np.random.choice(len(all_images), size=num_demo_images, replace=False)
demo_image_paths = [all_images[idx] for idx in demo_image_indexes]

print("Num demo images: ", num_demo_images)
print("Demo image paths: ")
for img in demo_image_paths:
    print("    " , img)

## Run Detection:

In [ ]:
opt.demo = list(map(str, demo_image_paths))
demo(opt)

In [ ]:
coco.imgToAnns[1274]